# Fix User Roles Foreign Key

The error `Key (user_id)=(...) is not present in table "users"` happens because your `user_roles` table is referencing a non-existent `public.users` table instead of the correct `auth.users` table where Supabase stores user accounts.

Since I cannot connect to your database directly (password authentication failed), please run the following SQL manually in your Supabase Dashboard.

1. Go to **Supabase Dashboard** > **SQL Editor**.
2. Click **New Query**.
3. Paste and run the following SQL:

```sql
-- Fix foreign key constraint on user_roles table
DO $$ 
DECLARE
    constraint_record RECORD;
BEGIN
    -- Drop existing foreign key constraint on user_id if it exists
    FOR constraint_record IN 
        SELECT tc.constraint_name 
        FROM information_schema.table_constraints AS tc 
        JOIN information_schema.key_column_usage AS kcu
          ON tc.constraint_name = kcu.constraint_name
          AND tc.table_schema = kcu.table_schema
        WHERE tc.table_name = 'user_roles' 
          AND kcu.column_name = 'user_id'
          AND tc.constraint_type = 'FOREIGN KEY'
    LOOP
        EXECUTE 'ALTER TABLE user_roles DROP CONSTRAINT ' || constraint_record.constraint_name;
    END LOOP;
END $$;

-- Add correct foreign key constraint to auth.users
ALTER TABLE user_roles
ADD CONSTRAINT user_roles_user_id_fkey
FOREIGN KEY (user_id)
REFERENCES auth.users(id)
ON DELETE CASCADE;
```
